注释数据分析

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.cm as cm

import ast
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# # 下载必要的资源，下载过了这里注释掉
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger_eng')

In [2]:
# 读预测正确数据
def get_comment_and_attribute_true():
    comment_word_list = []
    attribute_list = []
    
    with open('D:/BERT_learing/CCDP/gcb/Linelevel/comment_attribute_true_other.txt', 'r') as f:
        # 直接读txt  之后的 lines 的类型是 str  需要把str 变为 list
        lines = f.readlines()
        
        for i in range(0, len(lines), 2):
#             print(type(lines[i].strip())) # str
            # str  ->  list
            w_list = ast.literal_eval(lines[i].strip())
            comment_word_list.append(w_list)
#             print(type(w_list))          # list
            
            w_list = ast.literal_eval(lines[i+1].strip())
            attribute_list.append(w_list)
            
    return comment_word_list,attribute_list

comment_word_list_true,attribute_list_true = get_comment_and_attribute_true()

In [3]:
# 读预测出错误数据
def get_comment_and_attribute_false():
    comment_word_list = []
    attribute_list = []
    
    with open('D:/BERT_learing/CCDP/gcb/Linelevel/comment_attribute_false_other.txt', 'r') as f:
        # 直接读txt  之后的 lines 的类型是 str  需要把str 变为 list
        lines = f.readlines()
        
        for i in range(0, len(lines), 2):
#             print(type(lines[i].strip())) # str
            # str  ->  list
            w_list = ast.literal_eval(lines[i].strip())
            comment_word_list.append(w_list)
#             print(type(w_list))          # list
            
            w_list = ast.literal_eval(lines[i+1].strip())
            attribute_list.append(w_list)
            
    return comment_word_list,attribute_list

comment_word_list_false,attribute_list_false  = get_comment_and_attribute_false()

In [4]:
# true comment lenth
len(comment_word_list_true)

2140

In [5]:
len(comment_word_list_false)

999

In [6]:
# 把词恢复成句子 ， 对比一下  恢复成句子 和 不恢复成句子 有什么区别
def restore_CommentWord2Sentence(comment_WordList):
    comment_sentence = []
    for comment_number in comment_WordList:

        # 将列表内的 一个个单词 组合成 句子 
        sentence = ' '.join(comment_number)
        # 去除句子中多余的空格
        sentence = sentence.replace(' .', '.')
        comment_sentence.append(sentence)
        
    return comment_sentence

comment_sentence_true = restore_CommentWord2Sentence(comment_word_list_true)
comment_sentence_false = restore_CommentWord2Sentence(comment_word_list_false)

In [7]:
def get_WordTagging_attribution_for_word(comment_list):
    tagged_words_list = []
    for tag in comment_list:
        tagged_words = pos_tag(tag)
        tagged_words_list.append(tagged_words)
    
    return tagged_words_list

tagged_words_list_true = get_WordTagging_attribution_for_word(comment_word_list_true)
tagged_words_list_false = get_WordTagging_attribution_for_word(comment_word_list_false)

In [8]:
# 返回列表，列表内元素为元组，元组（comment_token , tagging , attribution）
def get_commentToken_tagging_attribution(tagged_token_list,attribute_num_list):
    commentToken_tagging_attribution = []
    for token, attribute in zip(tagged_token_list,attribute_num_list) :
        updated_tagged_words = [(word, tag, attr) for (word, tag), attr in zip(token, attribute)]
        commentToken_tagging_attribution.append(updated_tagged_words)
    
    # 去掉 标点符号    
    punctuations = {'.', ',', ';', ':', '(', ')', '[', ']', '{', '}', '!', '?', '``', "''"}    
    commentToken_tagging_attribution_new = [[(word, tag, score) for word, tag, score in element if word not in punctuations]\
                                              for element in commentToken_tagging_attribution]
    
    return commentToken_tagging_attribution_new

commentToken_tagging_attribution_true = get_commentToken_tagging_attribution(tagged_words_list_true,attribute_list_true)
commentToken_tagging_attribution_false = get_commentToken_tagging_attribution(tagged_words_list_false,attribute_list_false)

In [9]:
commentToken_tagging_attribution_true[:2]

[[('Check', 'NNP', 0.06615379478295874),
  ('whether', 'IN', -0.03490188473047647),
  ('any', 'DT', 0.008398054857961254),
  ('insertion', 'NN', -0.06151974836271891),
  ('or', 'CC', -0.09158247841946639),
  ('deletion', 'NN', -0.12764445125147736),
  ('actions', 'NNS', -0.12677520013384092),
  ('are', 'VBP', -0.3794704638235606),
  ('currently', 'RB', -0.4640624810731986),
  ('queued', 'VBN', -0.09435039837525831)],
 [('Returns', 'NNS', 0.2655297041988693),
  ('a', 'DT', -0.01890499521092679),
  ('list', 'NN', -0.07001438246228214),
  ('of', 'IN', -0.5202112280317644),
  ('the', 'DT', 0.030141012568886932),
  ('types', 'NNS', -0.384091389670993),
  ('of', 'IN', -0.140379900185534),
  ('the', 'DT', 0.06332042244370131),
  ('fields', 'NNS', -0.20527322536722495),
  ('in', 'IN', -0.05985471237610083),
  ('a', 'DT', -0.05718442779580334),
  ('given', 'VBN', 0.010592889714493372),
  ('struct', 'NN', 0.024502759521536216),
  ('type', 'NN', -0.022268223475810804)]]

In [10]:
commentToken_tagging_attribution_false[:2]

[[('Creates', 'NNS', 0.11458344952426519),
  ('elastic', 'JJ', -0.0006598843461709625),
  ('node', 'NN', 0.5052721633269703),
  ('as', 'IN', 0.33499630700560096),
  ('single', 'JJ', -0.05882543752274158),
  ('member', 'NN', 0.07800669048462289),
  ('of', 'IN', 0.15551597879825943),
  ('a', 'DT', 0.006186693042392226),
  ('cluster', 'NN', 0.02894811070161832)],
 [('Marshal', 'NNP', -0.0055072141676009625),
  ('the', 'DT', -0.0059921762078705655),
  ('aggregator', 'NN', -0.17225829392324568),
  ('values', 'NNS', -0.25226188444692055),
  ('of', 'IN', -0.07330020970083342),
  ('to', 'TO', -0.34171316211196634),
  ('a', 'DT', -0.2607253794253743),
  ('JSONArray', 'NNP', -0.3033713641602568),
  ('that', 'WDT', -0.26595895663153307),
  ('will', 'MD', -0.1358578565637935),
  ('later', 'RB', -0.12559919577615442),
  ('be', 'VB', 0.055265722922906366),
  ('aggregated', 'VBN', 0.008455093824031831),
  ('by', 'IN', -0.17996834856623517),
  ('master', 'NN', -0.0624221056658038)]]

In [11]:
def get_tagging_type_attribution(token_tagging_attribution):
    
    # 定义词性标签集合: 名词  动词  形容词 副词 连词 限定词 代词
    tag_groups = {
        'noun': {'NN', 'NNS', 'NNP', 'NNPS'},
        'verb': {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
        'adjective': {'JJ', 'JJR', 'JJS'},
        'adverb': {'RB', 'RBR', 'RBS'},
        'conjunction': {'CC', 'IN'},
        'determiner': {'DT', 'PDT', 'WDT'},
        'pronoun': {'PRP', 'PRP$', 'WP', 'WP$'}
    }
    
    # 根据字典映射，将子类别改为父类别
    mapping = {tag: key for key, tags in tag_groups.items() for tag in tags}
    
    # 此处为 commentToken_tagging_attribution 将 第二列 替换为 tag_groups 值
    token_tagging_attribution_for_Ftype = [[(token, mapping.get(tag, tag), value)for token,tag,value in elements] \
                                           for elements in token_tagging_attribution]
       
    return token_tagging_attribution_for_Ftype

token_tagging_attribution_true = get_tagging_type_attribution(commentToken_tagging_attribution_true)
token_tagging_attribution_false = get_tagging_type_attribution(commentToken_tagging_attribution_false)

In [12]:
list_true =    [('noun',           0.1),
                ('verb',           0.1),
                ('adjective' ,     0.3),
                ('adverb'   ,      0.15)]

In [13]:
# rawdata
type_list_true_1 = [('conjunction',0.08),
                    ('noun',0.08),
                    ('adjective',0.1)]
type_list_true_2 = [('conjunction',-0.08),
                    ('noun',-0.08),
                    ('adjective',-0.1)]

In [14]:
# W5
W_type_list_true_1 = [('noun',0.06),
                        ('verb',0.06),
                        ('adjective',0.1),
                        ('adverb',0.08)]
W_type_list_true_2 = [('noun',-0.06),
                        ('verb',-0.06),
                        ('adjective',-0.3),
                        ('adverb',-0.08)]

In [15]:
# iqr
iqr_type_list_true_1 = [('noun',0.05),
                        ('verb',0.04),
                        ('adjective',0.25),
                        ('adverb',0.04)]
iqr_type_list_true_2 = [('noun',-0.05),
                        ('verb',-0.04),
                        ('adjective',-0.25),
                        ('adverb',-0.04)]

In [16]:
type_list_false_1 = [
    ('noun',           0.085),
    ('adjective' ,     0.1),
    ('conjunction' ,   0.09),
]
type_list_false_2 = [
    ('noun',           -0.085),
    ('adjective' ,     -0.1),
    ('conjunction' ,   -0.09),
]

In [17]:
def filter_low_average(tagging_attribution,average_list_1,average_list_2):
    type_dict_1 = dict(average_list_1)
    type_dict_2 = dict(average_list_2)

    
    filtered_nested_statements = []
    indices = []
    
    for index, sublist in enumerate(tagging_attribution):
        if all(stmt_type in type_dict_1 and value > type_dict_1[stmt_type] for word, stmt_type, value in sublist if stmt_type in type_dict_1):
            filtered_nested_statements.append(sublist)
            indices.append(index)
        elif all(stmt_type in type_dict_2 and value < type_dict_2[stmt_type] for word, stmt_type, value in sublist if stmt_type in type_dict_2):
            filtered_nested_statements.append(sublist)
            indices.append(index)

    return filtered_nested_statements,indices

filtered_nested_statements_true,indices_true = filter_low_average(token_tagging_attribution_true,type_list_true_1,type_list_true_2)
filtered_nested_statements_false,indices_false = filter_low_average(token_tagging_attribution_false,type_list_false_1,type_list_false_2)

In [18]:
filtered_nested_statements_true[0]

[('Allow', 'noun', 0.6699906173766011),
 ('to', 'TO', -0.08686999499680402),
 ('print', 'verb', -0.005054440617196577),
 ('directives', 'noun', 0.5714329883230839)]

In [19]:
filtered_nested_statements_false[0]

[('Returns', 'noun', 0.25774796696266744)]

In [20]:
len(indices_true)

68

In [21]:
indices_true

[10,
 55,
 108,
 115,
 137,
 142,
 146,
 149,
 151,
 205,
 247,
 252,
 289,
 326,
 328,
 333,
 372,
 375,
 400,
 459,
 492,
 493,
 503,
 519,
 755,
 999,
 1105,
 1110,
 1165,
 1209,
 1234,
 1271,
 1290,
 1307,
 1323,
 1326,
 1371,
 1393,
 1436,
 1439,
 1486,
 1487,
 1491,
 1510,
 1569,
 1574,
 1585,
 1614,
 1670,
 1673,
 1681,
 1682,
 1725,
 1754,
 1799,
 1845,
 1873,
 1879,
 1905,
 1930,
 2007,
 2028,
 2050,
 2062,
 2099,
 2108,
 2117,
 2121]

In [22]:
len(indices_false)

38

In [23]:
indices_false

[3,
 6,
 15,
 38,
 43,
 80,
 112,
 124,
 127,
 229,
 235,
 243,
 263,
 311,
 322,
 327,
 562,
 576,
 640,
 663,
 668,
 670,
 690,
 714,
 719,
 743,
 745,
 769,
 787,
 810,
 844,
 861,
 874,
 917,
 922,
 942,
 946,
 960]

过滤后与linelevel取交集